In [ ]:
#all import statements
import heapq as hq
import copy
import time

In [ ]:
#defining the operators
operators = [[-1,0],[1,0],[0,-1],[0,1]]

In [ ]:
# find_blank takes the puzzle as input and returns the index where the blank tile is located

def find_blank(puzzle):
  dirs = [[-1,0],[1,0],[0,-1],[0,1]]
  valid_dirs = []
  for i in range(len(puzzle)):
    for j in range(len(puzzle[0])):
      if(puzzle[i][j]==0):
        return [i,j]

#Returns the valid directions (up,down,left,right) for the position of empty tile

def find_swap_dirs(blank_coords,puzzle):
  valid_dirs = []
  i = blank_coords[0]
  j = blank_coords[1]
  for k in range(len(operators)):
    co_ords = [i+operators[k][0],j+operators[k][1]]
    if(co_ords[0]<len(puzzle[0]) and co_ords[0]>=0 and co_ords[1]<len(puzzle[0]) and co_ords[1]>=0):
      valid_dirs.append(operators[k])
  return valid_dirs




In [ ]:
# Class Tree to keep track of all nodes, children and cost functions
class TreeNode:
  def __init__(self,data):
    self.data = data
    self.children = []
    self.parent = None
    self.G_n = 0
    self.H_n = 0
    self.F_n = 0
    pass
  def __lt__(self,other):
    return self.F_n < other.F_n
  def add_child(self,child):
    child.parent = self
    child.G_n =self.G_n + 1
    self.children.append(child)
    child.children = []




In [ ]:
# func to append all valid nodes into the heapq
def queueing_func(nodes,valid_nodes):
  for node in valid_nodes:
    if node.data not in visited_nodes:
      hq.heappush(nodes,node)





In [ ]:
# function to compute Misplaced tile heuristic
def misplaced_tile_heuristic(grid):
  node_no = 1
  counter = 0
  for i in range(len(grid)):
    for j in range(len(grid)):
      if(grid[i][j]!=node_no and grid[i][j]!=0):
        counter+=1
      node_no +=1
  return counter

In [ ]:
# function to compute Manhattan distance heuristic
def manhattan_heuristic(grid):
  n = len(grid)
  node = 1
  dist = 0
  for i in range(n):
    for j in range(n):
      y = (grid[i][j] -1)%3
      x = (grid[i][j] -1)//3
      if node != grid[i][j] and grid[i][j]!=0 :
        dist += abs(x-i) + abs(y-j)
      node += 1
  return dist


















In [ ]:
# Function to swap blank tile with any other valid tile
def swap_positions(matrix,blank_index,curr_index):
  matrix_list = list(matrix)
  matrix_list = [list(i) for i in matrix_list]
  temp = matrix_list[blank_index[0]][blank_index[1]]
  matrix_list[blank_index[0]][blank_index[1]] = matrix_list[curr_index[0]][curr_index[1]]
  matrix_list[curr_index[0]][curr_index[1]] = temp
  updated_matrix = tuple(matrix_list)
  updated_matrix = tuple(tuple(i) for i in updated_matrix)
  return updated_matrix


In [ ]:
# Function calls the valid_dirs function to find all valid directions in which empty tile can be swapped
# For all these directions the empty tile is swapped with valid tiles. This gives the children of the parent node.
# The child cost functions are updated and all the valid children nodes to be pushed into the queue are returned

def expand(node,operators,heuristic):
    matrix_updated = copy.deepcopy(node.data)
    blank_index = find_blank(matrix_updated)
    valid_dirs = find_swap_dirs(blank_index,node.data)

    valid_nodes = []
    for dir in valid_dirs:

      new_index = [blank_index[0] + dir[0],blank_index[1]+dir[1]]
      matrix_new = swap_positions(matrix_updated,blank_index,new_index)
      child = TreeNode(matrix_new)
      node.add_child(child)
      update_cost(child,heuristic)
      valid_nodes.append(child)
    return valid_nodes








In [ ]:
# This function is called to update the cost based on the heuristic value chosen
def update_cost(node,heuristic):
  if(heuristic==1):
    node.F_n = node.G_n
  elif(heuristic==2):
    node.H_n = misplaced_tile_heuristic(node.data)
    node.F_n = node.G_n + node.H_n
  else:
    node.H_n = manhattan_heuristic(node.data)
    node.F_n = node.G_n + node.H_n


In [ ]:
# The main function that returns the solution
def general_search(heuristic):
  # variable to keep track of expanded nodes
  nodes_expanded = 0
  nodes = []
  node = initial_state_node
  # updating the cost function based on heuristic in the input
  update_cost(node,heuristic)
  hq.heappush(nodes,node)
  #variable to keep track of maximum queue size
  max_queue_size = 1

  while(True):

    # no solution is found if queue becomes empty
    if(len(nodes)==0):
      return "failure"
    node = hq.heappop(nodes)

    max_queue_size = max(max_queue_size,len(nodes))
    #Check if node has already been visited or not

    if(node.data not in visited_nodes):
      visited_nodes.add(node.data)

      # Solution found when node's data is same as goal
      if(node.data == goal):
        print("nodes expanded:",nodes_expanded)
        print("max queue size:",max_queue_size)
        print("depth",node.G_n)
        # If solution is found return solution node

        return node
      nodes_expanded += 1
      # explore current node and add its children
      queueing_func(nodes,expand(node,operators,heuristic))








In [ ]:
# Sample test cases provided in the project guidelines
puzzles = [((1,2,3),(4,5,6),(7,8,0)),
           ((1,2,3),(4,5,6),(0,7,8)),
           ((1,2,3),(5,0,6),(4,7,8)),
           ((1,3,6),(5,0,2),(4,7,8)),
           ((1,3,6),(5,0,7),(4,8,2)),
           ((1,6,7),(5,0,3),(4,8,2)),
           ((7,1,2),(4,8,5),(6,3,0)),
           ((0,7,2),(4,6,1),(3,5,8))
           ]

In [ ]:
#Eexecution of all sample test cases with varying solution depths
# The result is found for all 3 algorithms
#maximum queue size, solution depth, nodes expanded and time required for execution have been computed
#for all sample testcases
co = 1
for puzzle in puzzles:
  heuristic_input = [1,2,3]
  for i in heuristic_input:
    print(co)
    print("puzzle {} is: {}".format(co, puzzle))
    if(i==1):
      print("Uniform cost Search")
    elif(i==2):
      print("Misplaced tile heuristic")
    else:
      print("Manhattan Heurisitc")
    start_time = time.time()
    initial_state_node = TreeNode(puzzle)
    goal = ((1,2,3),(4,5,6),(7,8,0))
    visited_nodes = set()
    print(initial_state_node.data)
    general_search(i)
    end_time = time.time()
    time_taken = end_time - start_time
    print("Time taken: {:.5f} sec".format(time_taken))
  co += 1


1
puzzle 1 is: ((1, 2, 3), (4, 5, 6), (7, 8, 0))
Uniform cost Search
((1, 2, 3), (4, 5, 6), (7, 8, 0))
nodes expanded: 0
max queue size: 1
depth 0
Time taken: 0.00018 sec
1
puzzle 1 is: ((1, 2, 3), (4, 5, 6), (7, 8, 0))
Misplaced tile heuristic
((1, 2, 3), (4, 5, 6), (7, 8, 0))
nodes expanded: 0
max queue size: 1
depth 0
Time taken: 0.00012 sec
1
puzzle 1 is: ((1, 2, 3), (4, 5, 6), (7, 8, 0))
Manhattan Heurisitc
((1, 2, 3), (4, 5, 6), (7, 8, 0))
nodes expanded: 0
max queue size: 1
depth 0
Time taken: 0.00012 sec
2
puzzle 2 is: ((1, 2, 3), (4, 5, 6), (0, 7, 8))
Uniform cost Search
((1, 2, 3), (4, 5, 6), (0, 7, 8))
nodes expanded: 5
max queue size: 5
depth 2
Time taken: 0.00076 sec
2
puzzle 2 is: ((1, 2, 3), (4, 5, 6), (0, 7, 8))
Misplaced tile heuristic
((1, 2, 3), (4, 5, 6), (0, 7, 8))
nodes expanded: 2
max queue size: 2
depth 2
Time taken: 0.00033 sec
2
puzzle 2 is: ((1, 2, 3), (4, 5, 6), (0, 7, 8))
Manhattan Heurisitc
((1, 2, 3), (4, 5, 6), (0, 7, 8))
nodes expanded: 2
max queue size

In [ ]:
#Solving 8 puzzle with custom input
# maximum queue size, solution depth, nodes expanded and time required for execution will be computed for user input
print("Welcome to my 8 puzzle solver!\n")
print("Enter a valid puzzle with 0 representing the blank\n")
print("Enter 1st row:")
row1 = input().split(" ")
row1 = [int(i) for i in row1]
print("Enter 2nd row:")
row2 = input().split(" ")
row2 = [int(i) for i in row2]
print("Enter 3rd row:")
row3 = input().split(" ")
row3 = [int(i) for i in row3]
input_list = [row1,row2,row3]
input_grid = tuple(tuple(i) for i in input_list)

print("Select Algorithm:")
print("Type 1 for Uniform Cost Search")
print("Type 2 for Misplaced Tile Heuristic")
print("Type 3 for Manhattan Distance Heuristic")

heurisitc = int(input())
start_time = time.time()
initial_state_node = TreeNode(input_grid)
goal = ((1,2,3),(4,5,6),(7,8,0))
visited_nodes = set()
start_time = time.time()
res=general_search(heurisitc)
end_time = time.time()
time_taken = end_time - start_time
time_taken = round(time_taken, 3)
print("Time taken: {:.3f} sec".format(time_taken))




Welcome to my 8 puzzle solver!

Enter a valid puzzle with 0 representing the blank

Enter 1st row:
1 2 3
Enter 2nd row:
4 5 6
Enter 3rd row:
7 0 8
Select Algorithm:
Type 1 for Uniform Cost Search
Type 2 for Misplaced Tile Heuristic
Type 3 for Manhattan Distance Heuristic
3
nodes expanded: 1
max queue size: 2
depth 1
Time taken: 0.003 sec
